In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
# Sample training data
sentences = [
    "I am learning Tensorflow ",
    "Tensorflow is a Deep Learning framework",
    "It is used to create Neural Netowrks"
]

In [3]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1

# Create input sequences and labels
input_sequences = []
for sentence in sentences:
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(tokenized_sentence)):
        n_gram_sequence = tokenized_sentence[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences for equal length
max_sequence_length = max(len(seq) for seq in input_sequences)
padded_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

In [4]:
# Create predictors and labels
X = padded_sequences[:, :-1]
y = tf.keras.utils.to_categorical(padded_sequences[:, -1], num_classes=total_words)

In [5]:
# Build the RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 50, input_length=max_sequence_length - 1),
    tf.keras.layers.SimpleRNN(100),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=100, verbose=2)

Epoch 1/100
1/1 - 1s - loss: 2.7551 - accuracy: 0.0000e+00 - 996ms/epoch - 996ms/step
Epoch 2/100
1/1 - 0s - loss: 2.7039 - accuracy: 0.0714 - 7ms/epoch - 7ms/step
Epoch 3/100
1/1 - 0s - loss: 2.6556 - accuracy: 0.2143 - 6ms/epoch - 6ms/step
Epoch 4/100
1/1 - 0s - loss: 2.6086 - accuracy: 0.5000 - 6ms/epoch - 6ms/step
Epoch 5/100
1/1 - 0s - loss: 2.5615 - accuracy: 0.5714 - 8ms/epoch - 8ms/step
Epoch 6/100
1/1 - 0s - loss: 2.5133 - accuracy: 0.5000 - 8ms/epoch - 8ms/step
Epoch 7/100
1/1 - 0s - loss: 2.4631 - accuracy: 0.5000 - 7ms/epoch - 7ms/step
Epoch 8/100
1/1 - 0s - loss: 2.4105 - accuracy: 0.5000 - 7ms/epoch - 7ms/step
Epoch 9/100
1/1 - 0s - loss: 2.3551 - accuracy: 0.5000 - 7ms/epoch - 7ms/step
Epoch 10/100
1/1 - 0s - loss: 2.2972 - accuracy: 0.5000 - 8ms/epoch - 8ms/step
Epoch 11/100
1/1 - 0s - loss: 2.2370 - accuracy: 0.5000 - 8ms/epoch - 8ms/step
Epoch 12/100
1/1 - 0s - loss: 2.1755 - accuracy: 0.5000 - 7ms/epoch - 7ms/step
Epoch 13/100
1/1 - 0s - loss: 2.1133 - accuracy: 0.42

In [8]:
import numpy as np

# Function to generate text based on seed text
def generate_text(seed_text, next_words, model, max_sequence_length):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]

        # Get the index of the word with the highest probability
        predicted_index = np.argmax(predicted_probs)

        # Convert the index back to the word
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break

        seed_text += " " + output_word

    return seed_text

In [10]:
generated_text = generate_text("Tensorflow", 5, model, max_sequence_length)
print("Generated Text:", generated_text)

Generated Text: Tensorflow is a deep learning framework
